In [1]:
def combine_datasets(features, target):
    """
    Combina las características y el objetivo en un solo DataFrame.

    Parameters:
    features (pd.DataFrame): DataFrame de características.
    target (pd.DataFrame): DataFrame del objetivo.

    Returns:
    pd.DataFrame: DataFrame combinado.
    """
    return pd.concat([features.reset_index(drop=True), target.reset_index(drop=True)], axis=1)

def remove_low_correlation_columns(data, target_column, threshold=0.1):
    """
    Elimina columnas con correlación baja respecto a la columna objetivo.

    Parameters:
    data (pd.DataFrame): DataFrame con las características y la columna objetivo.
    target_column (str): Nombre de la columna objetivo.
    threshold (float): Umbral de correlación.

    Returns:
    pd.DataFrame: DataFrame sin las columnas de baja correlación.
    list: Lista de columnas eliminadas.
    """
    correlation_matrix = data.corr()
    correlation_with_target = correlation_matrix[target_column].abs()
    columns_to_remove = correlation_with_target[correlation_with_target < threshold].index.tolist()
    
    if target_column in columns_to_remove:
        columns_to_remove.remove(target_column)

    return data.drop(columns=columns_to_remove), columns_to_remove


In [2]:
def kfold_target_encoding(features, target, encode_col, n_splits=5):
    # Crear una serie vacía para almacenar los valores codificados
    encoded_values = pd.Series(index=features.index, dtype='float64')
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    for train_idx, val_idx in kf.split(features):
        train_data, val_data = features.iloc[train_idx], features.iloc[val_idx]
        fraud_rate = target.iloc[train_idx].groupby(train_data[encode_col]).mean()
        # Utilizar fillna(0) para evitar problemas con valores no mapeados
        encoded_values.iloc[val_idx] = val_data[encode_col].map(fraud_rate).fillna(0)
    
    # Asignar los valores codificados a la nueva columna en features
    features[f'{encode_col}_encoded'] = encoded_values
    return features

In [3]:
'''

La función haversine calcula la distancia entre dos puntos en la 
superficie de la Tierra dados sus valores de latitud y longitud.

'''

def haversine(lat1, lon1, lat2, lon2):
    # Radio de la Tierra en kilómetros
    R = 6371
    
    # Convertir la diferencia de latitudes y longitudes de grados a radianes
    dlat = np.radians(lat2 - lat1)  # Diferencia de latitudes
    dlon = np.radians(lon2 - lon1)  # Diferencia de longitudes
    
    # Aplicar la fórmula del haversine para calcular la distancia
    a = (np.sin(dlat / 2) ** 2 +  # Cálculo del primer término de la fórmula
         np.cos(np.radians(lat1)) *  # Cálculo del coseno de la latitud 1
         np.cos(np.radians(lat2)) *  # Cálculo del coseno de la latitud 2
         np.sin(dlon / 2) ** 2)  # Cálculo del segundo término de la fórmula
    
    # Cálculo del ángulo central en radianes
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Calcular la distancia entre los dos puntos utilizando el radio de la Tierra
    distance = R * c
    
    return distance  # Retornar la distancia calculada

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold 
from sklearn.preprocessing import StandardScaler 
import os

In [5]:
os.chdir("D:/Users/Jose Castro/Desktop/FINAL NO COUNTRY/c21-47-ft-data-bi")

In [6]:
url = "data/intermediate/data_eda.csv"
data_proc = pd.read_csv(url)

In [7]:
columns_selec = ['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud', 'merch_zipcode']

In [8]:
data_proc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 38 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   trans_date_trans_time          1296675 non-null  object 
 1   cc_num                         1296675 non-null  int64  
 2   merchant                       1296675 non-null  object 
 3   category                       1296675 non-null  object 
 4   amt                            1296675 non-null  float64
 5   first                          1296675 non-null  object 
 6   last                           1296675 non-null  object 
 7   gender                         1296675 non-null  object 
 8   street                         1296675 non-null  object 
 9   city                           1296675 non-null  object 
 10  state                          1296675 non-null  object 
 11  zip                            1296675 non-null  int64  
 12  lat           

In [9]:
#Dejamos el dataset con las columnas iniciales
data_proc_copy = data_proc[columns_selec]
data_proc_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1296675 non-null  object 
 1   cc_num                 1296675 non-null  int64  
 2   merchant               1296675 non-null  object 
 3   category               1296675 non-null  object 
 4   amt                    1296675 non-null  float64
 5   first                  1296675 non-null  object 
 6   last                   1296675 non-null  object 
 7   gender                 1296675 non-null  object 
 8   street                 1296675 non-null  object 
 9   city                   1296675 non-null  object 
 10  state                  1296675 non-null  object 
 11  zip                    1296675 non-null  int64  
 12  lat                    1296675 non-null  float64
 13  long                   1296675 non-null  float64
 14  city_pop          

In [10]:
features = data_proc_copy.drop(columns=["is_fraud"],axis=1)
target = data_proc_copy.is_fraud

In [11]:
# Convertir la columna de fecha y hora de transacción a formato datetime
features['trans_date_trans_time'] = pd.to_datetime(features['trans_date_trans_time'], errors='coerce')

# Extraer componentes de la fecha y hora
features['year'] = features['trans_date_trans_time'].dt.year        # Año de la transacción
features['month'] = features['trans_date_trans_time'].dt.month      # Mes de la transacción
features['day'] = features['trans_date_trans_time'].dt.day          # Día del mes
features['day_of_week'] = features['trans_date_trans_time'].dt.dayofweek  # Día de la semana (0=lunes, 6=domingo)
features['hour'] = features['trans_date_trans_time'].dt.hour        # Hora de la transacción

# Convertir la columna de fecha de nacimiento a formato datetime
features['dob'] = pd.to_datetime(features['dob'])

# Calcular la edad a partir de la fecha de nacimiento
features['age'] = np.floor((pd.Timestamp.now() - features['dob']).dt.days / 365.25).astype("int64")

# Calcular la distancia entre las coordenadas de la transacción y las del comerciante
features['distance'] = features.apply(lambda row: haversine(row['lat'], row['long'], row['merch_lat'], row['merch_long']), axis=1)

# Calcular la media y el total gastado por cada número de tarjeta de crédito
features['amount_per_transaction'] = features.groupby('cc_num')['amt'].transform('mean')  # Promedio por transacción
features['amount_per_age'] = features['amt'] / features['age']
# Contar el número de transacciones por número de tarjeta de crédito
features['transaction_count'] = features.groupby('cc_num')['cc_num'].transform('count')

# Se crea columna relacion entre el monto y la frecuencia de uso
features['amount_frequency_ratio'] = features['amount_per_transaction'] / features['transaction_count']
# Se crea columna diferencia de monto respecto al promedio
features['amount_diff'] = features['amt'] - features.groupby('cc_num')['amt'].transform('mean')

features['total_spent'] = features.groupby('cc_num')['amt'].transform('sum')  # Total gastado


# Calcular la relación entre el número de transacciones y la edad
features['age_transaction_ratio'] = features['transaction_count'] / features['age']

# Calcular la interacción entre edad y monto de transacción
features['age_amount_interaction'] = features['age'] * features['amt']

# Volver a convertir la columna de fecha y hora (si es necesario)
features['trans_date_trans_time'] = pd.to_datetime(features['trans_date_trans_time'])

# Calcular días desde la última transacción
features['days_since_last_transaction'] = features.groupby('cc_num')['trans_date_trans_time'].diff().dt.days.fillna(0)

# Calcular el promedio de días entre transacciones
features['avg_days_between_transactions'] = features.groupby('cc_num')['days_since_last_transaction'].transform('mean')

# Calcular la frecuencia de estado y categoría
state_freq = features['state'].value_counts().to_dict()
features['state_freq'] = features['state'].map(state_freq)

category_freq = features['category'].value_counts().to_dict()
features['category_freq'] = features['category'].map(category_freq)

merchant_freq = features['merchant'].value_counts().to_dict()
features['merchant_freq'] = features['merchant'].map(merchant_freq)

# Aplicar KFold target encoding
columns_to_encode = ['job']
for col in columns_to_encode:
    if col in features.columns:
        features = kfold_target_encoding(features, target, col)
    else:
        print(f"La columna {col} no existe en features.")

# Calcular la frecuencia de categoría y comerciante en datos no fraudulentos
category_freq_non_fraud = features['category'].value_counts().to_dict()
features['category_freq_non_fraud'] = features['category'].map(category_freq_non_fraud)

merchant_freq_non_fraud = features['merchant'].value_counts().to_dict()
features['merchant_freq_non_fraud'] = features['merchant'].map(merchant_freq_non_fraud)

In [12]:
columns_option_A = ["cc_num", "merchant", "category", "first", "last", "street", "city", "city_pop", 
                    "city_pop_group", "zip", "lat", "long", "job", "dob", "unix_time", "merch_lat", 
                    "merch_long", "merch_zipcode", "age", "trans_date_trans_time", "trans_num"]

columns_option_B = ["cc_num", "merchant", "category", "first", "last", "street", "city", "city_pop", 
                    "city_pop_group", "zip", "lat", "long", "job", "dob", "unix_time", "merch_lat", 
                    "merch_long", "merch_zipcode", "age", "trans_date_trans_time", "state", 
                    "age_classification", "age_classification_transaction_ratio", "time_diff", 
                    "category_encoded", "merchant_encoded", "city_pop_group_encoded", "trans_num"]

columns_option_C = ["amt", "amount_per_transaction", "amount_per_age", "amount_diff", 
                    "amount_frequency_ratio", "age_amount_interaction"]

columns_option_D = ['amt', 'year', 'month', 'day', 'day_of_week', 'hour', 'distance',
                    'amount_per_transaction', 'total_spent', 'transaction_count',
                    'age_transaction_ratio', 'age_amount_interaction', "gender",
                    'days_since_last_transaction', 'avg_days_between_transactions',
                    'state_freq', 'category_freq', 'merchant_freq', 'job_encoded',
                    'category_freq_non_fraud', 'merchant_freq_non_fraud', 'gender_M',
                    'is_fraud']


def eliminar_columnas(data, opcion):
    # Definir opciones en un diccionario con el nombre y las columnas a eliminar o mantener
    opciones = {
        'A': ("Opción A", columns_option_A),
        'B': ("Opción B", columns_option_B),
        'C': ("Opción C (mantener solo estas columnas)", columns_option_C),
        'D': ("Opción D (mantener solo estas columnas)", columns_option_D)
    }
    
    # Validar la opción seleccionada
    if opcion in opciones:
        nombre_opcion, columns = opciones[opcion]
        print(f"Aplicando {nombre_opcion}")
    else:
        print("Opción no válida. Elige 'A', 'B', 'C', o 'D'.")
        return data  # Devuelve el DataFrame sin cambios si la opción no es válida
    
    # Aplicar según la opción
    if opcion in ['C', 'D']:
        # Mantener solo las columnas especificadas si existen en el DataFrame
        data = data[data.columns.intersection(columns)]
    else:
        # Eliminar columnas especificadas si existen en el DataFrame
        data = data.drop(columns=[col for col in columns if col in data.columns], errors='ignore')
    
    return data


In [13]:
result_A = eliminar_columnas(features, 'A')
result_B = eliminar_columnas(features, 'B')
result_C = eliminar_columnas(features,'C')
result_D = eliminar_columnas(features,'D')

Aplicando Opción A
Aplicando Opción B
Aplicando Opción C (mantener solo estas columnas)
Aplicando Opción D (mantener solo estas columnas)


In [14]:
categorical_columns = result_A.select_dtypes(include=['object']).columns
feautres_encoded_A = pd.get_dummies(result_A, columns=categorical_columns, drop_first=True)

combined_data_A = combine_datasets(feautres_encoded_A, target)
features_encoded_cleaned_A, removed_columns = remove_low_correlation_columns(combined_data_A, 'is_fraud')

In [15]:
categorical_columns = result_B.select_dtypes(include=['object']).columns
feautres_encoded_B = pd.get_dummies(result_B, columns=categorical_columns, drop_first=True)

combined_data_B = combine_datasets(feautres_encoded_B, target)
features_encoded_cleaned_B, removed_columns = remove_low_correlation_columns(combined_data_B, 'is_fraud')

In [16]:
categorical_columns = result_C.select_dtypes(include=['object']).columns
feautres_encoded_C = pd.get_dummies(result_C, columns=categorical_columns, drop_first=True)

combined_data_C = combine_datasets(feautres_encoded_C, target)
features_encoded_cleaned_C, removed_columns = remove_low_correlation_columns(combined_data_C, 'is_fraud')

In [17]:
categorical_columns = result_D.select_dtypes(include=['object']).columns
feautres_encoded_D = pd.get_dummies(result_D, columns=categorical_columns, drop_first=True)

combined_data_D = combine_datasets(feautres_encoded_D, target)
features_encoded_cleaned_D, removed_columns = remove_low_correlation_columns(combined_data_D, 'is_fraud')

In [18]:
#Se exporta el dataset modificado
features_encoded_cleaned_D.to_feather('data_final_0') # Tiene 5 columnas

features_encoded_cleaned_A.to_feather('data_final_1') # Tiene 6 columnas

features_encoded_cleaned_C.to_feather('data_final_2') # Tiene 7 columnas

combined_data_D.to_feather('data_final_3') # Tiene 22 columnas

combined_data_B.to_feather('data_final_4') # Tiene 25 Columnas